# Exploring Hacker News Posts

This project seeks to identify trends with Hacker News Posts. Hacker News is a site where user-submitted stories (a.k.a. posts) are voited and commented upon. We will be using an abridged version of the following dataset: https://www.kaggle.com/hacker-news/hacker-news-posts. The original version of the dataset has about 300,000 rows. We are using a dataset that is a random sample of 20,000 submissions. The 280,000 rows were removed because they did not receive any comments.

We will be looking at 2 main types of posts: Ask HN and Show HN. Ask HN posts are submitted when a user has a specific question they wish to ask the community. Show HN posts are submitted when a user would like to showcase a project, product, or some other interesting content to the community.

We will compare these two types of posts to determine:
1. Do Ask HN or Show HN receive more comments on average?
2. Do posts created at a certain time receive more comments on average?

## Importing the Data

In [1]:
# Put all imports here
from csv import reader
import datetime as dt

In [2]:
with open("hacker_news.csv") as hn_file:
    hn_content = reader(hn_file)
    hn = list(hn_content)
    
hn[:5]

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'],
 ['12224879',
  'Interactive Dynamic Video',
  'http://www.interactivedynamicvideo.com/',
  '386',
  '52',
  'ne0phyte',
  '8/4/2016 11:52'],
 ['10975351',
  'How to Use Open Source and Shut the Fuck Up at the Same Time',
  'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/',
  '39',
  '10',
  'josep2',
  '1/26/2016 19:30'],
 ['11964716',
  "Florida DJs May Face Felony for April Fools' Water Joke",
  'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/',
  '2',
  '1',
  'vezycash',
  '6/23/2016 22:20'],
 ['11919867',
  'Technology ventures: From Idea to Enterprise',
  'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429',
  '3',
  '1',
  'hswarna',
  '6/17/2016 0:01']]

## Removing the Header

We can see that in our hn object, the first item is the header. Let's remove the header and store in a variable so we can analyze our data.

In [3]:
headers = hn[0]
hn = hn[1:]

print("Header is:", headers)
print("\n")
print("First five rows are:", hn[:5])

Header is: ['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


First five rows are: [['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '

## Extracting only Ask HN and Show HN Posts

In [4]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    lower_title = row[1].lower()
    if lower_title.startswith('ask hn'):
        ask_posts.append(row)
    elif lower_title.startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)
        
print("Number of posts in ask_posts:", len(ask_posts))
print("Number of posts in show_posts:", len(show_posts))
print("Number of posts in other_posts:", len(other_posts))

Number of posts in ask_posts: 1744
Number of posts in show_posts: 1162
Number of posts in other_posts: 17194


## Average Number of Comments for Ask HN and Show HN Posts

In [5]:
total_ask_comments = 0
for row in ask_posts:
    total_ask_comments += float(row[4])
    
total_show_comments = 0
for row in show_posts:
    total_show_comments += float(row[4])
    
avg_ask_comments = total_ask_comments / len(ask_posts)
avg_show_comments = total_show_comments / len(show_posts)

print("Average number of comments for an Ask HN post:", avg_ask_comments)
print("Average number of comments for a Show HN post:", avg_show_comments)

Average number of comments for an Ask HN post: 14.038417431192661
Average number of comments for a Show HN post: 10.31669535283993


Based on the above findings, it looks like Ask HN posts receive almost 4 more comments on average. Ask HN posts on average receive 14.04 comments, while Show HN posts on average receive 10.32 comments.

Because of this, we will focus the rest of this project on Ask posts. Specifically, we will determine how the time of a post affects the number of comments it receives.

## Finding the Amount of Ask Posts and Comments by Hour Created

We will perform these two steps:
1. Calculate the number of ask posts created in each hour of the day, along with the number of comments received.
2. Calculate the average number of comments each ask post receives by hour created.

### Calculating Number of Ask Posts and Comments Per Hour

In [6]:
result_list = []
for post in ask_posts:
    created_at = dt.datetime.strptime(post[6], "%m/%d/%Y %H:%M")
    num_comments = int(post[4])
    result_list.append([created_at, num_comments])
    
# This will give us the number of ASK HN posts by hour.    
counts_by_hour = {}
# This will give us the number of comments on ASK HN posts by hour.
comments_by_hour = {}

for row in result_list:
    hour = row[0].hour
    if hour not in counts_by_hour:
        counts_by_hour[hour] = 1
    else:
        counts_by_hour[hour] += 1
    
    if hour not in comments_by_hour:
        comments_by_hour[hour] = row[1]
    else:
        comments_by_hour[hour] += row[1]
        
print("Posts By Hour:", counts_by_hour)
print("\n")
print("Comments By Hour:", comments_by_hour)

Posts By Hour: {9: 45, 13: 85, 10: 59, 14: 107, 16: 108, 23: 68, 12: 73, 17: 100, 15: 116, 21: 109, 20: 80, 2: 58, 18: 109, 3: 54, 5: 46, 19: 110, 1: 60, 22: 71, 8: 48, 4: 47, 0: 55, 6: 44, 7: 34, 11: 58}


Comments By Hour: {9: 251, 13: 1253, 10: 793, 14: 1416, 16: 1814, 23: 543, 12: 687, 17: 1146, 15: 4477, 21: 1745, 20: 1722, 2: 1381, 18: 1439, 3: 421, 5: 464, 19: 1188, 1: 683, 22: 479, 8: 492, 4: 337, 0: 447, 6: 397, 7: 267, 11: 641}


We now have two dictionaries. counts_by_hour gives us the number of ask posts in each hour. comments_by_hour gives us the number of comments on ask posts in each hour. We will now create a list of lists, where each element is a 2-item list, with the first item being the hour, and the second item being the average number of comments per post.

In [7]:
avg_by_hour = []
for hour in counts_by_hour:
    num_posts = counts_by_hour[hour]
    num_comments = comments_by_hour[hour]
    avg_comments_per_post = num_comments / num_posts
    avg_by_hour.append([hour, avg_comments_per_post])
    
print(avg_by_hour)

[[9, 5.5777777777777775], [13, 14.741176470588234], [10, 13.440677966101696], [14, 13.233644859813085], [16, 16.796296296296298], [23, 7.985294117647059], [12, 9.41095890410959], [17, 11.46], [15, 38.5948275862069], [21, 16.009174311926607], [20, 21.525], [2, 23.810344827586206], [18, 13.20183486238532], [3, 7.796296296296297], [5, 10.08695652173913], [19, 10.8], [1, 11.383333333333333], [22, 6.746478873239437], [8, 10.25], [4, 7.170212765957447], [0, 8.127272727272727], [6, 9.022727272727273], [7, 7.852941176470588], [11, 11.051724137931034]]


## Improving Readability of Results

We have a list that contains the average number of comments per Ask HN post for each hour. However, it is quite hard to read and understand. Let's finish by sorting the list of lists and printing the five highest values in a readable format.

Start by swapping the order of the elements so we can call the sorted function and sort by avg number of comments.

In [8]:
swap_avg_by_hour = []
for element in avg_by_hour:
    swap_avg_by_hour.append([element[1], element[0]])
    
print(swap_avg_by_hour)

[[5.5777777777777775, 9], [14.741176470588234, 13], [13.440677966101696, 10], [13.233644859813085, 14], [16.796296296296298, 16], [7.985294117647059, 23], [9.41095890410959, 12], [11.46, 17], [38.5948275862069, 15], [16.009174311926607, 21], [21.525, 20], [23.810344827586206, 2], [13.20183486238532, 18], [7.796296296296297, 3], [10.08695652173913, 5], [10.8, 19], [11.383333333333333, 1], [6.746478873239437, 22], [10.25, 8], [7.170212765957447, 4], [8.127272727272727, 0], [9.022727272727273, 6], [7.852941176470588, 7], [11.051724137931034, 11]]


In [9]:
sorted_swap = sorted(swap_avg_by_hour, reverse=True)
sorted_swap

[[38.5948275862069, 15],
 [23.810344827586206, 2],
 [21.525, 20],
 [16.796296296296298, 16],
 [16.009174311926607, 21],
 [14.741176470588234, 13],
 [13.440677966101696, 10],
 [13.233644859813085, 14],
 [13.20183486238532, 18],
 [11.46, 17],
 [11.383333333333333, 1],
 [11.051724137931034, 11],
 [10.8, 19],
 [10.25, 8],
 [10.08695652173913, 5],
 [9.41095890410959, 12],
 [9.022727272727273, 6],
 [8.127272727272727, 0],
 [7.985294117647059, 23],
 [7.852941176470588, 7],
 [7.796296296296297, 3],
 [7.170212765957447, 4],
 [6.746478873239437, 22],
 [5.5777777777777775, 9]]

In [10]:
print("Top 5 Hours for Ask Posts Comments")

Top 5 Hours for Ask Posts Comments


In [11]:
for element in sorted_swap[:5]:
    hour = str(element[1])
    hour_dt = dt.datetime.strptime(hour, "%H")
    formatted_hour = dt.datetime.strftime(hour_dt, "%H:%M")
    formatted_string = "{formatted_hour}: {avg_comments:.2f} average comments per post".format(
        formatted_hour=formatted_hour, avg_comments=element[0])
    print(formatted_string)

15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post


## Findings

Based on the above data, Ask HN posts that are posted during 15:00, 02:00, 20:00, 16:00, and 21:00 EST have the highest chance of receiving comments.

Let's explore the data a bit further.

## Do Show or Ask Posts Receive More Points On Average?

We will do a similar analysis on points now (num_points is index 3 in each row).

In [12]:
total_ask_points = 0
for row in ask_posts:
    total_ask_points += float(row[3])
    
total_show_points = 0
for row in show_posts:
    total_show_points += float(row[3])
    
avg_ask_points = total_ask_points / len(ask_posts)
avg_show_points = total_show_points / len(show_posts)

print("Average number of points per Ask HN post:", avg_ask_points)
print("Average number of points per Show HN post:", avg_show_points)


Average number of points per Ask HN post: 15.061926605504587
Average number of points per Show HN post: 27.555077452667813


It looks like Show HN posts receive a lot more points (almost 2x) on average than Ask HN posts. We will focus the next section on Show HN posts.

## Do Posts Created At Certain Times Receive More Points?

In [13]:
result_list = []
for post in show_posts:
    created_at = dt.datetime.strptime(post[6], "%m/%d/%Y %H:%M")
    num_points = int(post[3])
    result_list.append([created_at, num_points])
    
# This will give us the number of Show HN posts by hour.    
posts_by_hour = {}
# This will give us the number of points on Show HN posts by hour.
points_by_hour = {}

for row in result_list:
    hour = row[0].hour
    if hour not in posts_by_hour:
        posts_by_hour[hour] = 1
    else:
        posts_by_hour[hour] += 1
    
    if hour not in points_by_hour:
        points_by_hour[hour] = row[1]
    else:
        points_by_hour[hour] += row[1]
        
print("Posts By Hour:", posts_by_hour)
print("\n")
print("Points By Hour:", points_by_hour)

Posts By Hour: {14: 86, 22: 46, 18: 61, 7: 26, 20: 60, 5: 19, 16: 93, 19: 55, 15: 78, 3: 27, 17: 93, 6: 16, 2: 30, 13: 99, 8: 34, 21: 47, 4: 26, 11: 44, 12: 61, 23: 36, 9: 30, 1: 28, 10: 36, 0: 31}


Points By Hour: {14: 2187, 22: 1856, 18: 2215, 7: 494, 20: 1819, 5: 104, 16: 2634, 19: 1702, 15: 2228, 3: 679, 17: 2521, 6: 375, 2: 340, 13: 2438, 8: 519, 21: 866, 4: 386, 11: 1480, 12: 2543, 23: 1526, 9: 553, 1: 700, 10: 681, 0: 1173}


Let's calculate the average points per post by hour.

In [14]:
avg_points_by_hour = []
for hour in posts_by_hour:
    num_posts = posts_by_hour[hour]
    num_points = points_by_hour[hour]
    avg_points = num_points / num_posts
    avg_points_by_hour.append([avg_points, hour])
    
swapped_avg_points_by_hour = sorted(avg_points_by_hour, reverse=True)

sorted_avg_points_by_hour = []
for element in swapped_avg_points_by_hour:
    sorted_avg_points_by_hour.append([element[1], element[0]])
    
sorted_avg_points_by_hour

[[23, 42.388888888888886],
 [12, 41.68852459016394],
 [22, 40.34782608695652],
 [0, 37.83870967741935],
 [18, 36.31147540983606],
 [11, 33.63636363636363],
 [19, 30.945454545454545],
 [20, 30.316666666666666],
 [15, 28.564102564102566],
 [16, 28.322580645161292],
 [17, 27.107526881720432],
 [14, 25.430232558139537],
 [3, 25.14814814814815],
 [1, 25.0],
 [13, 24.626262626262626],
 [6, 23.4375],
 [7, 19.0],
 [10, 18.916666666666668],
 [9, 18.433333333333334],
 [21, 18.425531914893618],
 [8, 15.264705882352942],
 [4, 14.846153846153847],
 [2, 11.333333333333334],
 [5, 5.473684210526316]]

In [15]:
print("The 5 hours that have the highest number of points per Show HN post are:")

The 5 hours that have the highest number of points per Show HN post are:


In [16]:
for element in sorted_avg_points_by_hour[:5]:
    hour = str(element[0])
    hour_dt = dt.datetime.strptime(hour, "%H")
    formatted_hour = dt.datetime.strftime(hour_dt, "%H:%M")
    formatted_string = "{formatted_hour}: {avg_points:.2f} average points per post".format(
        formatted_hour=formatted_hour, avg_points=element[1])
    print(formatted_string)

23:00: 42.39 average points per post
12:00: 41.69 average points per post
22:00: 40.35 average points per post
00:00: 37.84 average points per post
18:00: 36.31 average points per post


Based on the above results, if a user wants to maximize the number of points they get on a Show HN post, they should post at 23:00, 12:00, 22:00, 00:00, or 18:00 EST.

## Comparing Avg Comments And Points to Other (non-Ask, non-Show) Posts

Now let's compare the average number of comments and points of Ask HN and Show HN posts to those of Other Posts.

In [17]:
num_other_comments = 0
num_other_points = 0
for post in other_posts:
    num_comments = int(post[4])
    num_points = int(post[3])
    num_other_comments += num_comments
    num_other_points += num_points
    
avg_other_comments = num_other_comments / len(other_posts)
avg_other_points = num_other_points / len(other_posts)
    
print("Average number of comments for Other Posts:", avg_other_comments)
print("Average number of comments for Ask HN Posts:", avg_ask_comments)
print("Average number of comments for Show HN Posts:", avg_show_comments)
print("\n")
print("Average number of points for Other Posts:", avg_other_points)
print("Average number of points for Show HN Posts:", avg_show_points)
print("Average number of points for Ask HN Posts:", avg_ask_points)

Average number of comments for Other Posts: 26.8730371059672
Average number of comments for Ask HN Posts: 14.038417431192661
Average number of comments for Show HN Posts: 10.31669535283993


Average number of points for Other Posts: 55.4067698034198
Average number of points for Show HN Posts: 27.555077452667813
Average number of points for Ask HN Posts: 15.061926605504587


It looks like Other Posts on average receive a lot more comments and points than either Ask HN or Show HN posts.

## Conclusion

We answered a series of questions in this project:

1. Do Ask HN or Show HN posts get more comments on average?
Ask HN posts receive almost 4 comments on average than Show HN posts.  
Average number of comments for an Ask HN post: 14.038417431192661  
Average number of comments for a Show HN post: 10.31669535283993  


2. What times should a user make an Ask HN post to get the most comments?
Here are the 5 hours in EST with the highest average number of comments per Ask HN post:  
15:00: 38.59 average comments per post  
02:00: 23.81 average comments per post  
20:00: 21.52 average comments per post  
16:00: 16.80 average comments per post  
21:00: 16.01 average comments per post  


3. Do Ask HN or Show HN posts get more points on average?  
Show HN posts receive almost twice as many points on average than Ask HN posts.  
Average number of points per Ask HN post: 15.061926605504587  
Average number of points per Show HN post: 27.555077452667813  


4. What times should a user make an Show HN post to get the most points?  
Here are the 5 hours in EST with the highest average number of points per Show HN post:  
23:00: 42.39 average points per post  
12:00: 41.69 average points per post  
22:00: 40.35 average points per post  
00:00: 37.84 average points per post  
18:00: 36.31 average points per post  


5. How do Other posts compare to Ask HN and Show HN posts when it comes to average number of comments and posts?
If a user wishes to get the most comments/points on his/her post, it would be best to stay away from Ask HN and Show HN posts, as Other Posts get about twice as many comments and points than either of the former options.  
Average number of comments for Other Posts: 26.8730371059672 
Average number of comments for Ask HN Posts: 14.038417431192661
Average number of comments for Show HN Posts: 10.31669535283993

Average number of points for Other Posts: 55.4067698034198
Average number of points for Show HN Posts: 27.555077452667813
Average number of points for Ask HN Posts: 15.061926605504587

Another trend we noticed with the time analysis is that the most active times on HackerNoon appear to be late night (21:00 - 2:00 EST). 6/10 of the top 10 hours for average number of comments and points per Ask HN and Show HN post occur in this time range.